In [2]:
# config.py

import os
import numpy as np
import tensorflow as tf
from google.colab import drive


# Mounting drive
# This will require authentication
drive.mount('/content/drive')


SEED = 42
EPOCHS = 100
BATCH_SIZE = 32
IMG_HEIGHT = 192
IMG_WIDTH = 256

input_shape = (IMG_HEIGHT, IMG_WIDTH, 3)

APP_PATH = '/content/drive/MyDrive/unik/cataract-mat-model'

def get_full_path(*pathes):
    path = ''.join(map(str, pathes))
    return os.path.join(APP_PATH, path)

# # cataract dataset
# IMG_ROOT = 'input/dataset/'
# IMG_DIR = [IMG_ROOT+'1_normal', 
#            IMG_ROOT+'2_cataract', 
#            IMG_ROOT+'2_glaucoma', 
#            IMG_ROOT+'3_retina_disease']

# ocular-disease-recognition dataset
OCU_ROOT = 'input/ODIR_5K/'
OCU_IMG_ROOT = 'input/ODIR_5K/Training_Images/'

# FULL_IMG_ROOT = get_full_path(IMG_ROOT)
FULL_OCU_ROOT = get_full_path(OCU_ROOT)
FULL_OCU_IMG_ROOT = get_full_path(OCU_IMG_ROOT)
# FULL_OCU_DATA_ROOT = get_full_path(FULL_OCU_ROOT, "data.xlsx")

# ocu_df = pd.read_excel(
#      FULL_OCU_DATA_ROOT,
#      engine='openpyxl',
# )


Mounted at /content/drive


In [4]:
# mist_activator.py

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import *


class Mish(tf.keras.layers.Layer):

    def __init__(self, **kwargs):
        super(Mish, self).__init__(**kwargs)
        self.supports_masking = True

    def call(self, inputs):
        return inputs * K.tanh(K.softplus(inputs))

    def get_config(self):
        base_config = super(Mish, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        return input_shape


def mish(x):
    return tf.keras.layers.Lambda(lambda x: x*K.tanh(K.softplus(x)))(x)


In [7]:
# predict_on_efficient_model.py

"""
  Need to be install
"""
# !pip install openpyxl

import os, glob, cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.utils import get_custom_objects

#from mish_activator import *
#from config import *


def seed_everything(seed):
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

seed_everything(SEED)

 
get_custom_objects().update({'mish': mish})

input_shape = (IMG_HEIGHT, IMG_WIDTH, 3)


def create_one_example(path, img_width, img_height):
    img = plt.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (img_width, img_height))
        
    return img

def normalize_example(img):
    return img[np.newaxis, :]/255.0


"""
	Loading custom model and predicting on separate example
"""

model_path = get_full_path('custom_model_with_activator.h5')

loaded_model = tf.keras.models.load_model(
      model_path,
      custom_objects={'mish': mish})

example_path = get_full_path(OCU_IMG_ROOT, '0_left.jpg')
example = create_one_example(example_path, IMG_WIDTH, IMG_HEIGHT)
example = normalize_example(example)

result = loaded_model.predict(example)
try:
  result_percent = result[0][1] * 100
except IndexError as e:
  print('Invalid data: {}'.format(e))

print('Custom Model\nDiagnosis: cataract with {:.2f}% probability'.format(result_percent))


Custom Model
Diagnosis: cataract with 69.18% probability
